In [1]:
import pickle
import numpy as np
import seaborn as sns
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [2]:
path = "/kaggle/input/dataset-chest/"

In [3]:
df_chest_filt = pd.read_pickle((path + "combined_chest_filtered.pkl"))

In [4]:
df_chest_filt.head(10)

,id,ACC_x,ACC_y,ACC_z,ECG,EMG,EDA,TEMP,RESP,label
214583,2.0,0.8914,-0.1102,-0.2576,0.030945,-0.003708,5.710983,29.083618,1.191711,1.0
214584,2.0,0.8926,-0.1086,-0.2544,0.033646,-0.014145,5.719376,29.122437,1.139832,1.0
214585,2.0,0.8930,-0.1094,-0.2580,0.033005,0.010208,5.706406,29.115234,1.141357,1.0
214586,2.0,0.8934,-0.1082,-0.2538,0.031815,0.012634,5.712509,29.126709,1.155090,1.0
214587,2.0,0.8930,-0.1096,-0.2570,0.030350,0.002060,5.727005,29.100861,1.133728,1.0
214588,2.0,0.8906,-0.1078,-0.2538,0.030350,-0.002792,5.707550,29.126709,1.136780,1.0
214589,2.0,0.8906,-0.1082,-0.2566,0.032639,-0.001968,5.715561,29.116699,1.115417,1.0
214590,2.0,0.8906,-0.1080,-0.2600,0.029572,0.007919,5.706406,29.106628,1.136780,1.0
214591,2.0,0.8872,-0.1062,-0.2586,0.026138,-0.007050,5.708313,29.121002,1.106262,1.0
214592,2.0,0.8862,-0.1068,-0.2616,0.027649,-0.013870,5.715561,29.108063,1.081848,1.0


In [5]:
df_chest_filt.describe()

,id,ACC_x,ACC_y,ACC_z,ECG,EMG,EDA,TEMP,RESP,label
count,2.320632e+07,2.320632e+07,2.320632e+07,2.320632e+07,2.320632e+07,2.320632e+07,2.320632e+07,2.320632e+07,2.320632e+07,2.320632e+07
mean,9.399516e+00,8.048063e-01,-3.151446e-02,-3.450945e-01,1.112485e-03,-2.954540e-03,4.601632e+00,3.377196e+01,5.366276e-02,1.636944e+00
std,4.705341e+00,1.298794e-01,1.077167e-01,2.643120e-01,2.708310e-01,1.401874e-02,3.451288e+00,1.315684e+00,3.615568e+00,7.533773e-01
min,2.000000e+00,2.345999e-01,-6.222000e-01,-1.798400e+00,-1.499405e+00,-6.101532e-01,4.451752e-01,2.804526e+01,-3.039398e+01,1.000000e+00
25%,5.000000e+00,7.290000e-01,-6.239998e-02,-5.446000e-01,-9.146118e-02,-9.841919e-03,1.989365e+00,3.354745e+01,-1.893616e+00,1.000000e+00
50%,9.000000e+00,8.554000e-01,-2.020001e-02,-3.234000e-01,-2.522278e-02,-2.609253e-03,3.719711e+00,3.412451e+01,-2.746582e-01,1.000000e+00
75%,1.400000e+01,8.974000e-01,2.380002e-02,-1.682000e-01,3.648376e-02,4.119873e-03,6.327057e+00,3.453427e+01,1.957703e+00,2.000000e+00
max,1.700000e+01,1.553400e+00,5.176001e-01,1.041800e+00,1.499954e+00,1.047180e+00,2.043343e+01,3.577805e+01,3.788605e+01,3.000000e+00


In [6]:
df_chest_filt.groupby(['id', 'label']).count()

ACC_x   ACC_y   ACC_z     ECG     EMG     EDA    TEMP    RESP
id   label                                                                
2.0  1.0    800800  800800  800800  800800  800800  800800  800800  800800
     2.0    430500  430500  430500  430500  430500  430500  430500  430500
     3.0    253400  253400  253400  253400  253400  253400  253400  253400
3.0  1.0    798000  798000  798000  798000  798000  798000  798000  798000
     2.0    447917  447917  447917  447917  447917  447917  447917  447917
     3.0    262500  262500  262500  262500  262500  262500  262500  262500
4.0  1.0    810601  810601  810601  810601  810601  810601  810601  810601
     2.0    444500  444500  444500  444500  444500  444500  444500  444500
     3.0    260400  260400  260400  260400  260400  260400  260400  260400
5.0  1.0    838600  838600  838600  838600  838600  838600  838600  838600
     2.0    451500  451500  451500  451500  451500  451500  451500  451500
     3.0    261800  261800  261800  261800  261800  261800  261800  261800
6.0  1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    455000  455000  455000  455000  455000  455000  455000  455000
     3.0    260400  260400  260400  260400  260400  260400  260400  260400
7.0  1.0    830200  830200  830200  830200  830200  830200  830200  830200
     2.0    448000  448000  448000  448000  448000  448000  448000  448000
     3.0    260401  260401  260401  260401  260401  260401  260401  260401
8.0  1.0    818300  818300  818300  818300  818300  818300  818300  818300
     2.0    469000  469000  469000  469000  469000  469000  469000  469000
     3.0    258999  258999  258999  258999  258999  258999  258999  258999
9.0  1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    451500  451500  451500  451500  451500  451500  451500  451500
     3.0    260400  260400  260400  260400  260400  260400  260400  260400
10.0 1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    507500  507500  507500  507500  507500  507500  507500  507500
     3.0    260400  260400  260400  260400  260400  260400  260400  260400
11.0 1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    476000  476000  476000  476000  476000  476000  476000  476000
     3.0    257600  257600  257600  257600  257600  257600  257600  257600
13.0 1.0    826001  826001  826001  826001  826001  826001  826001  826001
     2.0    464800  464800  464800  464800  464800  464800  464800  464800
     3.0    267400  267400  267400  267400  267400  267400  267400  267400
14.0 1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    472500  472500  472500  472500  472500  472500  472500  472500
     3.0    260401  260401  260401  260401  260401  260401  260401  260401
15.0 1.0    822500  822500  822500  822500  822500  822500  822500  822500
     2.0    480200  480200  480200  480200  480200  480200  480200  480200
     3.0    260400  260400  260400  260400  260400  260400  260400  260400
16.0 1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    471101  471101  471101  471101  471101  471101  471101  471101
     3.0    257600  257600  257600  257600  257600  257600  257600  257600
17.0 1.0    826700  826700  826700  826700  826700  826700  826700  826700
     2.0    506100  506100  506100  506100  506100  506100  506100  506100
     3.0    260400  260400  260400  260400  260400  260400  260400  260400

In [7]:
# Crear un modelo CNN para clasificación binaria para los datos recopilados de pecho
sampling_rate = 700  # Hz
window_time = 5  # segundos
sequence_length = sampling_rate * window_time
step_size = sequence_length // 2

# Extraer features y labels
X = df_chest_filt[['ACC_x', 'ACC_y', 'ACC_z', 'ECG', 'EMG', 'EDA', 'TEMP', 'RESP']].values
y = df_chest_filt['label'].values

# Crear segmentos con overlapping windows
segments = []
labels = []
for i in range(0, len(X) - sequence_length, step_size):
    segment = X[i:i + sequence_length]
    label = y[i + sequence_length - 1]  
    segments.append(segment)
    labels.append(label)

# Convertir a vector 
X_segments = np.array(segments)
y_segments = np.array(labels)
# One hot encoding, clasificar label 1(fase neutral) y 3(diversión) a un grupo(no estrés) y label 2(estrés) a otro
y_segments_binary = (y_segments != 2).astype(int)

# Redimensionar X_segments para adaptar a la dimensión de entrada para LSTM
num_features = X_segments.shape[2]
X_segments_reshaped = X_segments.reshape((X_segments.shape[0], X_segments.shape[1], num_features))

# Definir el input layer para todos los sensores
input_layer = Input(shape=(sequence_length, num_features))

# Función para crear un bloque convolucional de 1D 
def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

# Aplicar el bloque convolucional al input
output = conv_block(input_layer, filters=8, kernel_size=15, stride=2, pool_size=4, pool_stride=4)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dense(16, activation='relu')(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

# Generar el modelo
model_c_binary = Model(inputs=input_layer, outputs=output_layer)

# Imprimir por pantalla el resumen del modelo
model_c_binary.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3500, 8)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1743, 8)           968       
                                                                 
 max_pooling1d (MaxPooling1  (None, 435, 8)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 3480)              0         
                                                                 
 dense (Dense)               (None, 32)                111392    
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                             

In [8]:
# Normalización de datos
scaler = StandardScaler()
X_segments_normalized = scaler.fit_transform(X_segments_reshaped.reshape(-1, num_features)).reshape(X_segments_reshaped.shape)

# Dividir los datos para entrenamiento(80%) y para prueba(20%)
X_train, X_test, y_train, y_test = train_test_split(X_segments_normalized, y_segments_binary, test_size=0.2, random_state=42)

# Ejecutar el modelo
model_c_binary.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_c_binary.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_c_binary.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de pecho: {accuracy * 100:.2f}%')

Epoch 1/20
332/332 [==============================] - 10s 27ms/step - loss: 0.3613 - accuracy: 0.8153 - val_loss: 0.2049 - val_accuracy: 0.9178
Epoch 2/20
332/332 [==============================] - 8s 24ms/step - loss: 0.1547 - accuracy: 0.9406 - val_loss: 0.1639 - val_accuracy: 0.9280
Epoch 3/20
332/332 [==============================] - 8s 25ms/step - loss: 0.1073 - accuracy: 0.9595 - val_loss: 0.1515 - val_accuracy: 0.9412
Epoch 4/20
332/332 [==============================] - 8s 24ms/step - loss: 0.0822 - accuracy: 0.9729 - val_loss: 0.1260 - val_accuracy: 0.9566
Epoch 5/20
332/332 [==============================] - 8s 24ms/step - loss: 0.0672 - accuracy: 0.9774 - val_loss: 0.1837 - val_accuracy: 0.9370
Epoch 6/20
332/332 [==============================] - 8s 24ms/step - loss: 0.0500 - accuracy: 0.9834 - val_loss: 0.1161 - val_accuracy: 0.9634
Epoch 7/20
332/332 [==============================] - 8s 25ms/step - loss: 0.0505 - accuracy: 0.9827 - val_loss: 0.1010 - val_accuracy: 0.964

In [9]:
y_pred_binary = (model_c_binary.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)

# Imprimir los reesultados
print("Resultados del modelo de clasificación binaria para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {precision:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print('Matriz de confusión:')
print(conf_matrix)

83/83 [==============================] - 1s 7ms/step
Resultados del modelo de clasificación binaria para datos de pecho: 
F1-score: 0.9819
Precisión: 0.9857
Recall: 0.9781
Matriz de confusión:
[[ 797   26]
 [  40 1789]]


In [10]:
# Mejorar el modelo modificando algunos parámetros
sampling_rate = 700  # Hz
window_time = 5  # segundos
sequence_length = sampling_rate * window_time
step_size = sequence_length // 2

# Extraer features y labels
X = df_chest_filt[['ACC_x', 'ACC_y', 'ACC_z', 'ECG', 'EMG', 'EDA', 'TEMP', 'RESP']].values
y = df_chest_filt['label'].values

# Crear segmentos con overlapping windows
segments = []
labels = []
for i in range(0, len(X) - sequence_length, step_size):
    segment = X[i:i + sequence_length]
    label = y[i + sequence_length - 1]  
    segments.append(segment)
    labels.append(label)

# Convertir a vector 
X_segments = np.array(segments)
y_segments = np.array(labels)
# One hot encoding, clasificar label 1(fase neutral) y 3(diversión) a un grupo(no estrés) y label 2(estrés) a otro
y_segments_binary = (y_segments != 2).astype(int)

# Redimensionar X_segments para adaptar a la dimensión de entrada para LSTM
num_features = X_segments.shape[2]
X_segments_reshaped = X_segments.reshape((X_segments.shape[0], X_segments.shape[1], num_features))

# Definir el input layer para todos los sensores
input_layer = Input(shape=(sequence_length, num_features))

# Función para crear un bloque convolucional de 1D 
def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

# Aplicar el bloque convolucional al input
output = conv_block(input_layer, filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4)
output = conv_block(output, filters=32, kernel_size=10, stride=1, pool_size=2, pool_stride=2)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dropout(0.5)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

# Generar el modelo
model_c_binary = Model(inputs=input_layer, outputs=output_layer)

# Imprimir por pantalla el resumen del modelo
model_c_binary.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3500, 8)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 1743, 16)          1936      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 435, 16)           0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 426, 32)           5152      
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 213, 32)           0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 6816)              0   

In [11]:
# Normalización de datos
scaler = StandardScaler()
X_segments_normalized = scaler.fit_transform(X_segments_reshaped.reshape(-1, num_features)).reshape(X_segments_reshaped.shape)

# Dividir los datos para entrenamiento(80%) y para prueba(20%)
X_train, X_test, y_train, y_test = train_test_split(X_segments_normalized, y_segments_binary, test_size=0.2, random_state=42)

# Ejecutar el modelo
optimizer = Adam(lr=0.01)
model_c_binary.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_c_binary.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_c_binary.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de pecho: {accuracy * 100:.2f}%')

Epoch 1/20
332/332 [==============================] - 15s 41ms/step - loss: 0.4849 - accuracy: 0.6850 - val_loss: 0.3911 - val_accuracy: 0.7164
Epoch 2/20
332/332 [==============================] - 14s 43ms/step - loss: 0.3930 - accuracy: 0.7538 - val_loss: 0.3790 - val_accuracy: 0.7794
Epoch 3/20
332/332 [==============================] - 14s 42ms/step - loss: 0.3820 - accuracy: 0.7742 - val_loss: 0.4024 - val_accuracy: 0.7443
Epoch 4/20
332/332 [==============================] - 13s 38ms/step - loss: 0.3833 - accuracy: 0.7779 - val_loss: 0.3621 - val_accuracy: 0.7911
Epoch 5/20
332/332 [==============================] - 15s 45ms/step - loss: 0.3507 - accuracy: 0.7997 - val_loss: 0.3569 - val_accuracy: 0.7994
Epoch 6/20
332/332 [==============================] - 15s 46ms/step - loss: 0.3476 - accuracy: 0.8019 - val_loss: 0.3863 - val_accuracy: 0.7738
Epoch 7/20
332/332 [==============================] - 14s 43ms/step - loss: 0.3534 - accuracy: 0.7905 - val_loss: 0.3626 - val_accuracy:

In [12]:
y_pred_binary = (model_c_binary.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)

# Imprimir los reesultados
print("Resultados del modelo de clasificación binaria para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {precision:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print('Matriz de confusión:')
print(conf_matrix)

83/83 [==============================] - 1s 11ms/step
Resultados del modelo de clasificación binaria para datos de pecho: 
F1-score: 0.9943
Precisión: 0.9908
Recall: 0.9978
Matriz de confusión:
[[ 806   17]
 [   4 1825]]


In [13]:
# Generar un modelo CNN para la clasificaión de tres estados para los datos de pecho
# Modificar el modelo anterior para adaptar a la nueva funsión
X_segments_reshaped_2d = X_segments_reshaped.reshape(-1, num_features)

# Normalizar datos
scaler = StandardScaler()
X_segments_normalized_2d = scaler.fit_transform(X_segments_reshaped_2d)

# Redimensionar los datos normalizados a 3D
X_segments_normalized = X_segments_normalized_2d.reshape(X_segments_reshaped.shape)

# One hot encoding, num_classes = 4 porque los valores de label son de 1 a 3, tiene que ser mayor que 3 para que lo incluya
y_segments_one_hot = to_categorical(y_segments, num_classes=4) 

# Dividir datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_segments_normalized, y_segments_one_hot, test_size=0.2, random_state=42)

input_layer = Input(shape=(sequence_length, num_features))

def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

# Aplicar el bloque convolucional al input
output = conv_block(input_layer, filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4)
output = conv_block(output, filters=32, kernel_size=10, stride=1, pool_size=2, pool_stride=2)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dropout(0.5)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer para 3 clases, donde 4 es el num_classes
output_layer = Dense(4, activation='softmax')(x)

# Generar el modelo para clasificación de 3 estados
model_c_three = Model(inputs=input_layer, outputs=output_layer)

model_c_three.summary()
optimizer = Adam(lr=0.01)
model_c_three.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_c_three.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model_c_three.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación de tres estados para datos de pecho: {accuracy * 100:.2f}%')


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 3500, 8)]         0         
                                                                 
 conv1d_3 (Conv1D)           (None, 1743, 16)          1936      
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 435, 16)           0         
 g1D)                                                            
                                                                 
 conv1d_4 (Conv1D)           (None, 426, 32)           5152      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 213, 32)           0         
 g1D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 6816)              0   

In [14]:
y_pred_three = np.argmax(model_c_three.predict(X_test), axis=1)
y_true_three = np.argmax(y_test, axis=1)

f1 = f1_score(y_true_three, y_pred_three, average='weighted')
precision = precision_score(y_true_three, y_pred_three, average='weighted')
recall = recall_score(y_true_three, y_pred_three, average='weighted')

# Imprimir los reesultados
print("Resultados del modelo de clasificación de tres estados para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {precision:.4f}')
print(f'Recall: {recall:.4f}')

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_three, y_pred_three)
print('Matriz de confusión:')
print(conf_matrix)

83/83 [==============================] - 1s 11ms/step
Resultados del modelo de clasificación de tres estados para datos de pecho: 
F1-score: 0.9703
Precisión: 0.9718
Recall: 0.9698
Matriz de confusión:
[[1299    2   48]
 [   7  804   12]
 [   5    6  469]]
